# Clean Data

Ce fichier prend en entré des données dites `raw`, et les transformes en données utilisables pour le reste du projet. A travers ce projet, la logique est que seul ce fichier utilise les données dans `data_raw/`, et les autres fichiers n'utilisent que le dossier `data_clean/`, produit ici.

**Chargement des librairies**

In [37]:
import pandas as pd
import os
from datetime import datetime

**Chargement des données initiales, non filtrés**
|

In [38]:
df_bio = pd.read_pickle('../data_raw/df_bio.pkl')
df_condition = pd.read_pickle('../data_raw/df_condition.pkl')
df_dedup_deterministic = pd.read_pickle('../data_raw/df_dedup_deterministic.pkl')
df_dedup_proba = pd.read_pickle('../data_raw/df_dedup_proba.pkl')
df_note = pd.read_pickle('../data_raw/df_note.pkl')
df_person = pd.read_pickle('../data_raw/df_person.pkl')
df_visit = pd.read_pickle('../data_raw/df_visit.pkl')
 
input_dir = "../data_raw/"
output_dir = '../data_clean'

os.makedirs(output_dir, exist_ok=True)

**Clean de la base de données df_person**

In [39]:
# Le CDM est identique pour tous les patients, on le supprime
assert(len(df_person['cdm_source'].unique()) == 1)
df_person_clean = df_person.drop(columns=['cdm_source'])

# On supprime les patients sans date de naissance, ou avec une date de naissance trop ancienne (incoherent)
df_person_clean = df_person_clean[df_person_clean['birth_datetime'] > '1920-01-01']

# On adopte une unique convention pour le genre
df_person_clean['gender_source_value'] = df_person_clean['gender_source_value'].replace({'f': 'female', 'm': 'male'})
assert(len(df_person_clean.gender_source_value.unique()) == 2)

# On supprime les hommes de l'étude
df_person_clean = df_person_clean[df_person_clean['gender_source_value'] == 'female']
assert(len(df_person_clean.gender_source_value.unique()) == 1)

# On sauvegarde les données nettoyées dans un répertoire spécifique
output_path = os.path.join(output_dir, 'df_person.pkl')
df_person_clean.to_pickle(output_path)

**Suppression des patients dédupliqués**

In [40]:
df_person_dedup_det = pd.merge(
    df_person_clean,
    df_dedup_deterministic,
    on='person_id',
    how='outer'
)
df_person_dedup_det['unique_person_id'] = df_person_dedup_det['unique_person_id'].fillna(df_person_dedup_det['person_id'])
df_person_dedup_det = df_person_dedup_det.drop_duplicates(['unique_person_id'], keep='first')

# Fonction générique, pour tester différents seuils de probabilité facilement
def deduplicate_proba(df_person, df_dedup_proba, score):
    df_dedup_proba_score = df_dedup_proba[df_dedup_proba['prob'] > score]
    df_person_dedup_proba = pd.merge(df_person, df_dedup_proba_score, on='person_id', how='outer')
    df_person_dedup_proba['unique_person_id'] = df_person_dedup_proba['unique_person_id'].fillna(df_person_dedup_proba['person_id'])
    df_person_dedup_proba = df_person_dedup_proba.drop_duplicates(['unique_person_id'], keep='first')
    return df_person_dedup_proba

df_person_dedup_proba_90 = deduplicate_proba(df_person_clean, df_dedup_proba, score=0.90)
df_person_dedup_proba_20 = deduplicate_proba(df_person_clean, df_dedup_proba, score=0.20)

In [41]:
print("Original patient data:")
print(f"Unique patients: {df_person_clean['person_id'].nunique()}")

print('\nDeterministic deduplication:')
print(f"Unique patients: {df_person_dedup_det['unique_person_id'].nunique()}")

print('\nProbabilistic deduplication (threshold 0.90):')
print(f"Unique patients: {df_person_dedup_proba_90['unique_person_id'].nunique()}")

print('\nProbabilistic deduplication (threshold 0.20):')
print(f"Unique patients: {df_person_dedup_proba_20['unique_person_id'].nunique()}")

Original patient data:
Unique patients: 975

Deterministic deduplication:
Unique patients: 967

Probabilistic deduplication (threshold 0.90):
Unique patients: 959

Probabilistic deduplication (threshold 0.20):
Unique patients: 953


In [42]:
df_person_dedup_proba_90.to_pickle(os.path.join(output_dir, 'df_person.pkl'))

df_person_dedup = df_person_dedup_proba_90

In [43]:
# Utilisation de la fonction pour mettre à jour les identifiants uniques dans df_visit et df_condition, pour n'utiliser que les identifiants uniques
def update_with_unique_id(df_visit, df_person_dedup):
    df_updated = pd.merge(
        df_visit,
        df_person_dedup[['person_id', 'unique_person_id']],
        on='person_id',
        how='left'
    )

    df_updated['unique_person_id'] = df_updated['unique_person_id'].fillna(df_updated['person_id'])
    return df_updated

df_visit_dedup = update_with_unique_id(df_visit, df_person_dedup)
df_condition_dedup = update_with_unique_id(df_condition, df_person_dedup)

In [44]:
def finalize_person_id(df):
    df = df.drop(columns=['person_id'], errors='ignore')
    df = df.rename(columns={'unique_person_id': 'person_id'})
    return df

df_person_final = finalize_person_id(df_person_dedup)
df_visit_final = finalize_person_id(df_visit_dedup)
df_condition_final = finalize_person_id(df_condition_dedup)

df_person_final.to_pickle(os.path.join(output_dir, 'df_person.pkl'))
df_visit_final.to_pickle(os.path.join(output_dir, 'df_visit.pkl'))
df_condition_final.to_pickle(os.path.join(output_dir, 'df_condition.pkl'))

**Réflexion sur les autres bases de données**

In [45]:
df_visit = pd.read_pickle(os.path.join(output_dir, 'df_visit.pkl'))
df_visit = df_visit[df_visit['person_id'].isin(df_person_final['person_id'])]
pd.to_pickle(df_visit, os.path.join(output_dir, 'df_visit.pkl'))

df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_note = df_note[df_note['visit_occurrence_id'].isin(df_visit['visit_occurrence_id'])]
pd.to_pickle(df_note, os.path.join(output_dir, 'df_note.pkl'))

df_condition = pd.read_pickle(os.path.join(output_dir, 'df_condition.pkl'))
df_condition = df_condition[df_condition['person_id'].isin(df_person_final['person_id'])]
pd.to_pickle(df_condition, os.path.join(output_dir, 'df_condition.pkl'))

df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_bio = df_bio[df_bio['visit_occurrence_id'].isin(df_visit['visit_occurrence_id'])]
pd.to_pickle(df_bio, os.path.join(output_dir, 'df_bio.pkl'))


**Création de la table `df_facteur_risque` pour stocker les différents facteurs de risque relevés**

In [46]:
df_facteur_risque = df_person_final.copy()

# Determine si une personnes est indiqué comme ayant un cancer du sein dans la base de donnée
def a_le_cancer_sein(person_id, df_condition):
    cancer_sein_codes = {"C50", "C500", "C501", "C502", "C503", "C504"}
    patient_conditions = df_condition[df_condition['person_id'] == person_id]
    return any(str(code) in cancer_sein_codes for code in patient_conditions['condition_source_value'])

def bois_alcool(person_id, df_condition):
    # On considère que le patient a un problème d'alcool si il a une condition liée à l'alcool
    alcool_codes = {"T51"}
    patient_conditions = df_condition[df_condition['person_id'] == person_id]
    return any(str(code) in alcool_codes for code in patient_conditions['condition_source_value'])

df_bio['person_id'] = df_bio['visit_occurrence_id'].map(df_visit.set_index('visit_occurrence_id')['person_id'])

# Ajouter l'IMC a la table df_facteur_risque
for person_id in df_facteur_risque['person_id'].unique():
    target = df_bio[df_bio['person_id'] == person_id].copy()
    bmi_only = target[target['concept_source_value'] == "bmi"]
    imc = bmi_only['transformed_value'].values[0]
    df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id, 'imc'] = imc

# Dernière date de visite d'un patient
def get_last_visit_date(person_id, df_visit):
    visites = df_visit[df_visit['person_id'] == person_id]

    return visites['visit_start_datetime'].max()

# Calculer l'âge de chaque patient à partir de sa date de naissance et de la dernière visite ou de la date de décès
for person_id in df_facteur_risque['person_id'].unique():
    row = df_person_final[df_person_final['person_id'] == person_id]

    birth = row['birth_datetime'].values[0]
    death = row['death_datetime'].values[0] if not pd.isna(row['death_datetime'].values[0]) else None

    end_date = pd.to_datetime(death) if death else pd.to_datetime(get_last_visit_date(person_id, df_visit))
    age = (end_date - pd.to_datetime(birth)).days / 365.25

    df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id, 'age'] = age

for person_id in df_facteur_risque['person_id'].unique():
    has_cancer = a_le_cancer_sein(person_id, df_condition)
    df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id, 'cancer_sein'] = has_cancer

    alcool_consomation = bois_alcool(person_id, df_condition)
    df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id, 'alcool'] = alcool_consomation

# Suppressions des colonnes inutiles, du a la manière d'on on copie les person_id
df_facteur_risque.drop(columns=['birth_datetime', "death_datetime", "gender_source_value", "prob"], inplace=True)

pd.to_pickle(df_facteur_risque, os.path.join(output_dir, 'df_facteur_risque.pkl'))